In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import json
file = open('./../r_addresses.json')
wallet = json.load(file)

In [27]:
# Variables
from key import ECPubKey,ECKey

alicePrivateKey = ECKey().set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)
alicePubKeyBytes = alicePrivateKey.get_pubkey().get_bytes(False)
alicePubKey = ECPubKey().set(alicePubKeyBytes)
aliceSecret = ECKey().set(0x5031d70f63885182c6aaebcf0f9efc36d9faef330f25bb0df8bedd34d888ffb7)

bobPrivateKey = ECKey().set(bytes.fromhex("13e4f9204068a232f23fe6305bb6c1fcb6d2c6e53347956a7fbaf2953737ab4c"))
bobPubKeyBytes = bobPrivateKey.get_pubkey().get_bytes(False)
bobPubKey = ECPubKey().set(bobPubKeyBytes)
bobSecret = ECKey().set(0x8a41964da3b35d642a733b5539f323c9c300c48919ae6dfbd7a107a0f82fc189)

In [12]:
# 1. Alice Transaction -> BTC to BOB
from key import ECPubKey,ECKey
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness,ser_uint256
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash, CScript, OP_1, OP_CHECKMULTISIG, OP_0,OP_CHECKSIG, OP_2
import sys
sys.path.append("..")
from ecc import PrivateKey

prev_tx_hx = int("7c4e6957c0b39023414d60ec21a4a4194347d6d6f2353c6b1cabbcc284b71ef3", 16)
prv_amount=int(100*100000000)
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 1

# p=ECKey()
# p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)

# target_public_key_bytes=bytes.fromhex(wallet[12]["public key"])

target_amount = int(99.999*100000000)

tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index))

# target_public_key=ECPubKey().set(target_public_key_bytes)
# mast_scripts = [ CScript([target_public_key.get_bytes(), OP_CHECKSIG]) ]
mast_scripts = [ CScript([OP_2, alicePubKeyBytes, bobPubKeyBytes, OP_2, OP_CHECKMULTISIG])];

locking_script, tweak, control_map = taproot_construct(bobPubKey, mast_scripts)

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.vin=[tx_in]
tx.vout=[tx_outs]
tx.calc_sha256()
tx_sig_hash = SegwitV0SignatureHash(prv_script, tx, 0, 0x01, prv_amount)
tx_input_0_wit = CTxInWitness([alicePrivateKey.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'), alicePubKeyBytes])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx1_outs = tx.vout
t1 = tweak
cmp1 = control_map


01000000000101f31eb784c2bcab1c6b3c35f2d6d6474319a4a421ec604d412390b3c057694e7c0100000000ffffffff01605d0a540200000022512010f4e15e394f1ff52b06d1bb84da69c1529089ef0498e82b8a04296f1bd7228e0248304502210088db33933d49a326d29107ec3b27dd276eb4a55649d49a949d14bff90cc19fbd0220417f46303a842fa3313d3fc29bf76f44608adfe338a4d4ffae6bcf9a0b5242e4012103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee00000000


In [17]:
# 2. ALICE REVOKE TRANSACTION
from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes

taproot_locking_tx_id = int("6ca1cb6a524c195b81fa48f0f84f06f203f49e47ce62de9e718d6e7c4391f9db", 16)
taproot_locking_tx_index = 0

target_amount = int(99.998*100000000)

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
day1 = 101010001
tx_in_0.nSequence = (1 << 22) | day1;
tx_ins = [tx_in_0]

mast_scripts = [ CScript([OP_2, alicePubKeyBytes, bobPubKeyBytes, OP_2, OP_CHECKMULTISIG])];

locking_script, tweak, control_map = taproot_construct(bobPubKey, mast_scripts)

tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp1)[0])
tx_sign_0 = alicePrivateKey.sign_schnorr(tx_sig_hash_0)
tx_sign_1 = bobPrivateKey.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, tx_sign_1, list(cmp1)[0], cmp1[list(cmp1)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx2_outs = tx.vout
t2 = tweak
cmp2 = control_map



01000000000101dbf991437c6e8d719ede62ce479ef403f2064ff8f048fa815b194c526acba16c0000000000514a450601c0d608540200000022512010f4e15e394f1ff52b06d1bb84da69c1529089ef0498e82b8a04296f1bd7228e0440977fefe5a706a50bd04dffaef52b38364d253d044aca00a8ce83e9edb32b1a9620918819d3dbce2b5fb311562119c35ec53fc3a11f933f37f7ff155e9e3804c740076054b776ec76e1edc271ee6450a63f4ff70bbdc940bb34e0214d33ed3528c857208331980e47accc1f6c36732581930b0e7be3682e23f23c19a0393cc8fca847522103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee2103728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1952ae21c1728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000


In [23]:
# 3. ALICE REFUND2 TRANSACTION

from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes

taproot_locking_tx_id = int("6ca1cb6a524c195b81fa48f0f84f06f203f49e47ce62de9e718d6e7c4391f9db", 16)
taproot_locking_tx_index = 0

target_amount = int(99.997*100000000)

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
day1 = 101010001
tx_in_0.nSequence = (1 << 22) | day1;
tx_ins = [tx_in_0]

target_b32_address = wallet[0]["address"]
locking_public_key_hash = bytes(decode_segwit_address("bcrt", target_b32_address)[1])
locking_script = get_p2wpkh_script(locking_public_key_hash)
tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp2)[0])
tx_sign_0 = alicePrivateKey.sign_schnorr(tx_sig_hash_0)
tx_sign_1 = bobPrivateKey.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, tx_sign_1, aliceSecret, list(cmp2)[0], cmp1[list(cmp2)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())



01000000000101dbf991437c6e8d719ede62ce479ef403f2064ff8f048fa815b194c526acba16c0000000000514a45060120500754020000001600147d7143a28a3b836e3043f3ba5dbcdc4ee29b74570540892f5201ffa1023d6b4973ae3953eaa361d5b8ba883b0032c8cd14f67d218e3685ab80ebc3b8ab41c2944a998d6e281bc2150d5ad00998423d775e853c08fe08402c53c7afca79ae0a24d8dffee8c3302c5b3062d058ebdd9986864ad83dc6a41b0967e593de75c5ce1c011e87a9fafcc8dd4ba0e0e0d4ddb814545b6b9abdcd3d04123ac45e47522103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee2103728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1952ae21c1728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000


In [24]:
# 4. BOB REFUND TRANSACTION

from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes

taproot_locking_tx_id = int("6ca1cb6a524c195b81fa48f0f84f06f203f49e47ce62de9e718d6e7c4391f9db", 16)
taproot_locking_tx_index = 0

target_amount = int(99.997*100000000)

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
day1 = 101010001
tx_in_0.nSequence = (1 << 22) | (day1 << 1);
tx_ins = [tx_in_0]

target_b32_address = wallet[12]["address"]
locking_public_key_hash = bytes(decode_segwit_address("bcrt", target_b32_address)[1])
locking_script = get_p2wpkh_script(locking_public_key_hash)
tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp2)[0])
tx_sign_0 = alicePrivateKey.sign_schnorr(tx_sig_hash_0)
tx_sign_1 = bobPrivateKey.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, tx_sign_1, list(cmp2)[0], cmp1[list(cmp2)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())



01000000000101dbf991437c6e8d719ede62ce479ef403f2064ff8f048fa815b194c526acba16c0000000000a2944a0c012050075402000000160014fce7aed87fbb8b90029dee7fe3b5ed28466347a504403365ff2504885baa99f816c41aff88a886cd4634c59c8d0f2e205864b186c5aec230fa1eade96bbce18349f0371002c96c463772bc801079c8724a26bb25ff5d408993c429a81f2e5931c4a8f6a80558c0b73596850e12723acba481cfab9c4adb6f960e07e7191b8d62c362889c3740bd40d06c88635bdcf75d78c510c3ffea2847522103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee2103728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1952ae21c1728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000


In [28]:
# 5. BOB Transaction -> BTC to Alice
from key import ECPubKey,ECKey
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness,ser_uint256
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash, CScript, OP_1, OP_CHECKMULTISIG, OP_0,OP_CHECKSIG, OP_2
import sys
sys.path.append("..")
from ecc import PrivateKey

prev_tx_hx = int("7c4e6957c0b39023414d60ec21a4a4194347d6d6f2353c6b1cabbcc284b71ef3", 16)
prv_amount=int(100*100000000)
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 1

# p=ECKey()
# p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)

# target_public_key_bytes=bytes.fromhex(wallet[12]["public key"])

target_amount = int(99.999*100000000)

tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index))

# target_public_key=ECPubKey().set(target_public_key_bytes)
# mast_scripts = [ CScript([target_public_key.get_bytes(), OP_CHECKSIG]) ]
mast_scripts = [ CScript([OP_2, aliceSecret.get_pubkey().get_bytes(False), bobSecret.get_pubkey().get_bytes(False), OP_2, OP_CHECKMULTISIG])];

locking_script, tweak, control_map = taproot_construct(bobPubKey, mast_scripts)

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.vin=[tx_in]
tx.vout=[tx_outs]
tx.calc_sha256()

tx_sig_hash = SegwitV0SignatureHash(prv_script, tx, 0, 0x01, prv_amount)
tx_input_0_wit = CTxInWitness([bobPrivateKey.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'), bobPubKeyBytes])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())


01000000000101f31eb784c2bcab1c6b3c35f2d6d6474319a4a421ec604d412390b3c057694e7c0100000000ffffffff01605d0a54020000002251204d90f87234aabdef90cb1fda02c78d725f5fb81fc77b3961a5a06e2b8b678e690247304402203c3794fd8952c11e078e51d1ef9c9fa5c425ca94bd905ef3655f9872564ef0f402205aca2ddf12946c2057fdd65bce0888aa423264c8b8eb7df2ee467bef18673a81012103728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000


In [25]:
# 6. BOB SUCCESS TRANSACTION

from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes

taproot_locking_tx_id = int("6ca1cb6a524c195b81fa48f0f84f06f203f49e47ce62de9e718d6e7c4391f9db", 16)
taproot_locking_tx_index = 0

target_amount = int(99.998*100000000)

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
tx_ins = [tx_in_0]

target_b32_address = wallet[12]["address"]
locking_public_key_hash = bytes(decode_segwit_address("bcrt", target_b32_address)[1])
locking_script = get_p2wpkh_script(locking_public_key_hash)
tx_out_0 = CTxOut(target_amount, locking_script)
tx_outs = [tx_out_0]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp1)[0])
tx_sign_0 = alicePrivateKey.sign_schnorr(tx_sig_hash_0)
tx_sign_1 = bobPrivateKey.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, tx_sign_1, bobSecret, list(cmp1)[0], cmp1[list(cmp1)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())



01000000000101dbf991437c6e8d719ede62ce479ef403f2064ff8f048fa815b194c526acba16c0000000000ffffffff01c0d6085402000000160014fce7aed87fbb8b90029dee7fe3b5ed28466347a505409e1eeee6af90dfd22fccbfb5881a3f330228c3b72aa243746eea8d867507f7b8099a4cb7ae359e6f763bcdc6609967a289db3a2f02a3b7fb4f242e4f634047e2406f0dc70a4be9fe85930e3a1e83f74546fc75aa3d695d3e22cd7fa5970cb5420cdede383171b934388a2a0918c3356f1a87d635ca63a1ff1b921965f3a42157860689d97908d97047522103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee2103728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1952ae21c1728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000


In [40]:
from script import taproot_tree_helper
internal_pub_key = ECPubKey().set(bytes.fromhex("187791b6f712a8ea41c8ecdd0ee77fab3e85263b37e1ec18a3651926b3a6cf27"))
locking_script, tweak, cmp = taproot_construct(internal_pub_key, [ (0xc0, CScript(bytes.fromhex("20d85a959b0290bf19bb89ed43c916be835475d013da4b362117393e25a48229b8ac")) )])

print(taproot_tree_helper([ (0xc0, CScript(bytes.fromhex("20d85a959b0290bf19bb89ed43c916be835475d013da4b362117393e25a48229b8ac")) )])[1].hex())
print(internal_pub_key.tweak_add(tweak).get_bytes().hex())
print(locking_script.hex())
print(tweak.hex())
print(cmp[list(cmp)[0]].hex())



5b75adecf53548f3ec6ad7d78383bf84cc57b55a3127c72b9a2481752dd88b21
147c9c57132f6e7ecddba9800bb0c4449251c92a1e60371ee77557b6620f3ea3
5120147c9c57132f6e7ecddba9800bb0c4449251c92a1e60371ee77557b6620f3ea3
cbd8679ba636c1110ea247542cfbd964131a6be84f873f7f3b62a777528ed001
c1187791b6f712a8ea41c8ecdd0ee77fab3e85263b37e1ec18a3651926b3a6cf27


In [120]:
bye = b'\t\\\xd0\xa9XTK\x88\xd7\xacX\xa2\xf2\x9c\xcdI\xc9VZ\x9f\x7f\xe7\xaa\x96m\xf4\x19\xbf}\xa0\xa5+'
print(bye.hex())

bye2 = b'1\x96\x9f\xa4\x08\xe5\x1b/\xec1!\xebjM*\xc8\x02\xa9\x87\xaa\x89\nHdw\xb72\x0e\xb5\x107z'
print(bye2.hex())

print(SECP256K1.lift_x(int.from_bytes(pub_key, 'big')))

print("43273777186443610501550240681962765976649068965092868303218065941808905780180")

095cd0a958544b88d7ac58a2f29ccd49c9565a9f7fe7aa966df419bf7da0a52b
31969fa408e51b2fec3121eb6a4d2ac802a987aa890a486477b7320eb510377a
(51816362344230832880300216262509008239902364748236324249889473786728335112729, 4162054245958954304649435371766049053427989041172356308662552853458952477290, 1)
51816362344230832880300216262509008239902364748236324249889473786728335112729
